In [ ]:
!pip install PyPDF2 python-docx spacy wordcloud

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 19.9 MB/s eta 0:00:00


In [7]:
from PyPDF2 import PdfReader

reader=PdfReader("Test.pdf")
for i, page in enumerate(reader.pages):
   text=page.extract_text()
   print(f"Page {i+1}")
   print(text)


Page 1
Panchayati Raj System in Kerala – A  
Model of Local Governance
Submitted By:
Gaurang Bhatia - RA2411003012036
Narasimha B - RA2411003012039
Dhanush - RA2411003011975
Rahul Prasanth R - RA2411003012035
Vasu Moradiya - RA2411003012524
Dev Raj - RA2411003011970
Ayush Kashyap - RA2411003011992
Tushar - RA2411003012025
Animesh - RA2411003011981
Page 2
Introduction
Kerala is one of India’s leading states when it comes to local governance. This report 
highlights how the state successfully implemented the Panchayati Raj system 
through strong decentralization. The focus is on empowering people, improving 
services, and encouraging development that begins at the village level.
What is Decentralization?
Decentralization is the process of shifting power and responsibilities from central or  
state governments to local self-government institutions. It allows for more public 
participation and makes administration more responsive and effective. In a 
decentralized setup, local bodies get a